In [ ]:
#use sqlite3 to store intermediate query results
import sqlite3

# Step 2: Establish a connection to the in-memory database
conn = sqlite3.connect(':memory:')

# Step 3: Perform database operations
cursor = conn.cursor()


In [ ]:
from google.colab import drive
drive.mount('drive')

In [ ]:
import pandas as pd

In [ ]:
# configure connection to BigQuery
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

project = 'bis638-zlsy' # replace <project id> with your own project id
location = 'US' #this is the default location
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

In [ ]:
sqlpad = ('''SELECT p.subject_id, p.gender, p.dob, p.dod, p.dod_hosp, p.dod_ssn, date_diff(a.admittime, p.DOB, YEAR) AS age,
       a.hadm_id, a.admittime, a.dischtime, a.deathtime, a.admission_type, a.admission_location, a.discharge_location, a.insurance, a.language, a.religion, a.marital_status, a.ethnicity, a.edregtime, a.edouttime, a.diagnosis, a.hospital_expire_flag, a.has_chartevents_data,
       d.icd9_code,d.seq_num,
       n.chartdate, n.charttime, n.storetime, n.category, n.description, n.cgid, n.iserror, n.text
FROM `physionet-data.mimiciii_clinical.patients` p
INNER JOIN `physionet-data.mimiciii_clinical.admissions` a ON p.subject_id = a.subject_id
INNER JOIN `physionet-data.mimiciii_clinical.diagnoses_icd` d ON a.hadm_id = d.hadm_id
INNER JOIN `physionet-data.mimiciii_notes.noteevents` n ON p.subject_id = n.subject_id
WHERE d.icd9_code LIKE '585';
''')
df = client.query(sqlpad).to_dataframe()


In [ ]:
sqlchart1 = '''SELECT a.*, b.LABEL, b.ABBREVIATION,b.DBSOURCE,d.ICD9_CODE
FROM `physionet-data.mimiciii_clinical.chartevents` a
inner join `physionet-data.mimiciii_clinical.d_items` b on a.ITEMID = b.ITEMID
INNER JOIN `physionet-data.mimiciii_clinical.admissions` c ON a.hadm_id = c.hadm_id
INNER JOIN `physionet-data.mimiciii_clinical.diagnoses_icd` d ON c.hadm_id = d.hadm_id
WHERE d.icd9_code LIKE '585' and b.LINKSTO = "chartevents"'''
df4 = client.query(sqlchart1).to_dataframe()

In [ ]:
sqllab_2 = '''SELECT lab.*,
items.LABEL, items.FLUID, items.CATEGORY, items.LOINC_CODE
FROM `physionet-data.mimiciii_clinical.labevents` lab
INNER JOIN `physionet-data.mimiciii_clinical.d_labitems` items ON lab.itemid = items.itemid
INNER JOIN `physionet-data.mimiciii_clinical.admissions` c ON lab.hadm_id = c.hadm_id
INNER JOIN `physionet-data.mimiciii_clinical.diagnoses_icd` d ON c.hadm_id = d.hadm_id
WHERE d.icd9_code LIKE '585';'''

df2 = client.query(sqllab_2).to_dataframe()

In [ ]:

df.to_csv('data_padn.csv', index=False)

!cp data_lab.csv "drive/My Drive/"




df2.to_csv('data_lab.csv', index=False)

!cp data_lab.csv "drive/My Drive/"


df4.to_csv('data_chart.csv', index=False)

!cp data_chart.csv "drive/My Drive/"
